In [13]:
from io import StringIO
import pandas as pd
import numpy as np
import operator

sessions = pd.read_parquet('../data/interim/sessions_new_sqr_flex_not_filtered.parquet')
user_profiles = pd.read_parquet('../data/raw/flex_perfiles_usuario.parquet')

 

data = StringIO("""
1610,2448,,,,,,,,
2445,2988,,,,,,,,
1497,1751,1752,1753,,,,,,
1522,1523,1799,,,,,,,
1798,2397,2992,2996,3905,,,,,
1595,1596,1626,1813,1814,,,,,
8026,8036,8226,8282,,,,,,
7991,7992,8049,,,,,,,
8031,8040,8052,8191,8328,,,,,
8032,8041,8053,8054,8329,,,,,
5519,5547,5554,5638,6133,8024,8027,8037,,
5525,5546,5548,5550,5639,6135,7845,8039,,
5710,5814,6964,,,,,,,
5707,5715,5747,6963,,,,,,
5746,5897,6055,6269,,,,,,
5745,5896,6021,6022,6054,,,,,
5844,5869,5878,5998,6060,6102,6611,6844,6845,7592
7593,6061,6843,6610,5843,6612,5845,5868,,
5888,5996,,,,,,,,
5956,6005,6006,,,,,,,
5964,6275,,,,,,,,
5966,6274,,,,,,,,
6014,6024,6078,,,,,,,
6039,6314,,,,,,,,
6115,6119,6335,,,,,,,
6228,6378,6606,6905,,,,,,
6629,6848,,,,,,,,
6249,6261,7854,,,,,,,
6497,6502,8200,,,,,,,
6569,6570,6756,,,,,,,
6614,6868,7719,8225,,,,,,
6613,6869,7718,,,,,,,
6628,6692,6753,6755,,,,,,
""")
profiles = pd.read_csv(data, header=None)
sessions['alejandro_profile'] = None

 

def f(r):
    sessions.loc[sessions['idPerfil'].isin(r.values[~np.isnan(r.values)]), 'alejandro_profile'] = r.name
    
def format_pressures(p):
    return ''.join(operator.itemgetter(0, 2, *range(4, 8))(p))

 

profiles.apply(f, axis=1)
aux = sessions[~sessions['alejandro_profile'].isna()].copy()
aux['pressures'] = aux['idPerfil'].apply(lambda x: format_pressures(user_profiles[user_profiles['idPerfiles'] == x].iloc[0].presiones))
aux['temporal_profile'] = aux.sort_values('fechaInicio').groupby(
    'alejandro_profile')['pressures'].apply(lambda x: (x.shift() != x).rolling(len(x), min_periods=1).sum())

In [15]:
tabla1 = aux.groupby(['alejandro_profile', 'temporal_profile']).agg({'pressures': np.max, 'sqr': np.mean})

In [16]:
tabla1.reset_index()['alejandro_profile'].value_counts()[tabla1.reset_index()['alejandro_profile'].value_counts() > 1].index

Int64Index([5, 12, 27, 25, 24, 31, 10, 11, 13, 16, 17, 18, 19, 20, 21, 23], dtype='int64')

In [17]:
tabla2 = tabla1.reset_index()[tabla1.reset_index()['alejandro_profile'].isin([5, 12, 27, 25, 24, 31, 10, 11, 13, 16, 17, 18, 19, 20, 21, 23])]

In [19]:
tabla2['configuracion2'] = tabla2['pressures']

In [20]:
tabla2

,alejandro_profile,temporal_profile,pressures,sqr,configuracion2
0,5,1.0,022321,54.806296,022321
1,5,2.0,000000,46.869593,000000
2,5,3.0,012221,38.217778,012221
3,5,4.0,122222,48.436435,122222
4,5,5.0,012221,38.918137,012221
8,10,1.0,122332,65.133793,122332
9,10,2.0,012232,35.892428,012232
10,11,1.0,123222,45.692213,123222
11,11,2.0,122222,43.571443,122222
12,12,1.0,122222,61.258148,122222


In [21]:
tabla2['magnitud'] = tabla2.groupby('alejandro_profile')['sqr'].apply(lambda x: (x - x.shift()))
tabla2['configuracion1'] = tabla2.groupby('alejandro_profile')['pressures'].apply(lambda x: (x.shift()))

In [22]:
tabla2['sqr_1'] = tabla2.groupby('alejandro_profile')['sqr'].apply(lambda x: (x.shift()))
tabla2['sqr_2'] = tabla2['sqr']

In [23]:
tabla3 = tabla2.dropna()[['alejandro_profile', 'configuracion1', 'sqr_1', 'configuracion2', 'sqr_2', 'magnitud']]
tabla3['mejora'] = tabla3['magnitud'] > 0

In [24]:
tabla3

,alejandro_profile,configuracion1,sqr_1,configuracion2,sqr_2,magnitud,mejora
1,5,022321,54.806296,000000,46.869593,-7.936703,False
2,5,000000,46.869593,012221,38.217778,-8.651816,False
3,5,012221,38.217778,122222,48.436435,10.218657,True
4,5,122222,48.436435,012221,38.918137,-9.518297,False
9,10,122332,65.133793,012232,35.892428,-29.241365,False
11,11,123222,45.692213,122222,43.571443,-2.120769,False
13,12,122222,61.258148,122211,50.195206,-11.062943,False
14,12,122211,50.195206,022211,53.724454,3.529248,True
16,13,123431,64.122135,123331,65.364942,1.242807,True
19,16,012211,43.487271,122221,48.983882,5.496611,True
